### 1. Processing the SNP's data from the variant calling step
 a) Load in snps list
 
 b) Generate dataframe for each dataset

 c) Comine the metadata (Assign category for each sample)
 
 
 
## African Countries  Dataset
## Eastern Africa
Kenya (81)- (Downloaded, QC, Variant Calling)

Tanzania (84) (Downloaded, QC, Variant Calling)

## Western Africa
Nigeria (279) (Downloaded, QC, Variant Calling)

Senegal (39)  (Downloaded, QC, Variant Calling)
## Southern Africa
South Africa (44)- (Downloaded, QC, Variant Calling)

Mauritius (11)
## Nothern Africa
Tunisia (88)

Morocco (11) Processing ongoing

In [3]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from complexcgr import CGR
from pathlib import Path
# Instantiate class CGR
cgr = CGR()

def get_last_name(file_path):
    return Path(file_path).stem

def preprocessing(snp_list,df_bio,df_sra,country):
    # Step1: merge SNPs of all samples
    with open(snp_list, "r") as f:
        sample_files = f.read().splitlines()

    try:
        # Read the data from the first sample
        first_data = pd.read_csv(sample_files[0], sep=" ", header=None)
        first_data.columns = ["genomic_position", "REF", get_last_name(sample_files[0])]
        # Drop the REF column
        first_data = first_data.drop(columns=["REF"])

        # Start with first_data as the merged data
        merge_data = first_data

        # add other samples and merge them one by one
        for sample in sample_files[1:]:
            try:
                pd.read_csv(sample, sep=" ", header=None)
#                 print(sample)
                sample_data = pd.read_csv(sample, sep=" ", header=None)
                column_name = get_last_name(sample)
                sample_data.columns = ["genomic_position", "REF", column_name]
                # Drop the REF column
                sample_data = sample_data.drop(columns=["REF"])
                # merge data on genomic positions
                merge_data = merge_data.merge(sample_data, on="genomic_position", how="outer")
            except:
                continue

        merge_data.set_index("genomic_position", inplace=True)
        # Tranpose the df (reverse) to have samples as rows and genomic positions as columns
        merge_data = merge_data.T
        #replace nan with merged data
#        df=df.merge_data("N")
        df=merge_data.fillna("N")
        
        df=df.reset_index()
        df=df.rename(columns={"index":"sra_accession"})
        
        # Step 2: Remove constant columns
        non_constant_cols = df.columns[df.nunique() > 1]
        df_constant_dropped = df[non_constant_cols]
        df_constant_dropped
        df_constant_dropped.insert(1, "country", country)
        # read metadata
        df_bio=pd.read_table(df_bio)
        df_sra=pd.read_table(df_sra,header=None)
        #map sra to biosample
        df_sra_biosample=pd.concat([df_bio,df_sra], axis=1)
         # concat the sra and df
        df_sra_biosample.dropna(inplace=True)
        df_country=pd.concat([df_sra_biosample,df],axis=1)
        df_country
        
        df_country.to_csv(f"../data/{country}_processed.csv")

        # Move the return statement outside the loop, after all samples have been processed and merged.
        return df_constant_dropped

    except Exception as e:
        print("Error:", str(e))
        return None


# Call the preprocessing function and store the result in the 'df' variable
#     merge_data = merge_data.fillna("N")
#     merge_data = merge_data.replace({"A": 1, "G": 2, "C": 3, "T": 4, "N": 0})
#     merge_data = merge_data.rename(columns={"V1": "POS"})
#     tinput = merge_data.T
#     tinput.columns = tinput.iloc[0]
#     tinput = tinput.iloc[1:]
#     value = len(sample_files) // 5 * 4
#     tinput4 = tinput[tinput["Num"] < value]
#     tinput5 = tinput4.T
#     tinput5.to_csv("snp_input.csv", index=False)
#     # FCGR Encoding using kaos package
#     data = pd.read_csv("snp_input.csv")
#     data = data.replace({0: "N", 1: "A", 2: "G", 3: "C", 4: "T"})

#     for i in range(900):
#         cgr_list = cgr(data[str(i)], seq_base=["N", "A", "G", "C", "T"], res=200)
#         cgr_matrix = cgr_list["matrix"]
#         cgr_matrix.to_csv(sample_files[i], index=False)


In [7]:
def combine_metadata(df_snp,df_bio,df_sra,country):
        # read metadata
        df_bio=pd.read_table(df_bio)
        df_bio=df_bio[["BioSample", "Isolation source"]]
        df_sra=pd.read_table(df_sra,header=None,)
        df_sra.columns = ["BioSample", "sra_accession"]
#         df_snp.insert(1, "country", country)


#         print(df_bio.head())
        #map sra to biosample
        df_sra_biosample = pd.merge(df_bio, df_sra, on="BioSample", how="inner")
#         df_sra_biosample=pd.merge([df_bio,df_sra], on="BioSample")
#         print(df_sra.head())
         # concat the sra and df
        df_sra_biosample.dropna(inplace=True)
#         print(df_sra_biosample)
        df_country = pd.merge(df_sra_biosample, df_snp, on="sra_accession", how="inner")

        df_country
        
        df_country.to_csv(f"../data/{country}_processed.csv",index=False)

        # Move the return statement outside the loop, after all samples have been processed and merged.
        return df_country

In [99]:
df_kenya = preprocessing(snp_list="../data/kenya_snp.lists",
                         df_bio="../accessions/kenya/kenya.tsv",
                         df_sra="../pipeline/run-2023-08-11-salmonellaml/kenya_biosamples.txt",
                         country="kenya")

Error: [Errno 2] No such file or directory: '../data/SRR10502163.snp'


In [94]:
# df_tz = preprocessing(snp_list="../data/tz_snps/",
#                       df_bio="../work/5a/c62ab92992b24ec6c72c7c94c7903d/tanzania_biosamples.txt",
#                       df_sra="../wo",
#                       country="tanzania")

In [2]:
df_nigeria_paultry = preprocessing(snp_list="../data/nigeria_paultry/file_paths.txt",
                      df_bio="../accessions/nigeria/filtered_nigeria_nigeria_poultry_data.tsv",
                      df_sra="../work/c9/b58aedeb74c3574e9abd924de894b5/sra_filtered_nigeria_nigeria_poultry_data_biosamples.txt",
                      country="nigeria")
df_nigeria_paultry

genomic_position,sra_accession,country,27,40,576,646,681,765,793,796,...,4486464,4745998,4746010,4746025,4746046,4746058,4746064,4746067,4746068,4746070
0,ERR4181890,nigeria,A,A,A,T,C,T,C,T,...,N,N,N,N,N,N,N,N,N,N
1,ERR4181891,nigeria,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
2,ERR4181892,nigeria,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
3,ERR4181893,nigeria,A,N,N,T,N,N,C,T,...,N,N,N,N,N,N,N,N,N,N
4,ERR4181896,nigeria,A,N,A,T,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,SRR9994267,nigeria,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
80,SRR9994287,nigeria,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
81,SRR9994447,nigeria,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
82,SRR9998283,nigeria,N,N,N,T,C,T,C,T,...,N,N,N,N,N,N,N,N,N,N


In [76]:
df_south_africa = preprocessing(snp_list="../data/snp.lists",
                      df_bio="../accessions/south_africa/south_africa.tsv",
                      df_sra="../work/29/9d00016990fe8d0f623ba6840b6282/sra_south_africa_biosamples.txt",
                      country="south_africa")
df_south_africa

genomic_position,sra_accession,country,480,534,646,765,793,796,1125,1242,...,4562901,4562904,4562905,4562908,4562913,4562916,4562919,4585971,4607670,4815481
0,SRR14434432,south_africa,G,C,T,T,C,T,T,T,...,N,N,N,N,N,N,N,N,N,N
1,SRR14434433,south_africa,G,C,T,T,C,T,T,T,...,N,N,N,N,N,N,N,N,N,N
2,SRR14434434,south_africa,G,C,T,T,C,T,T,T,...,N,N,N,N,N,N,N,N,N,N
3,SRR14434436,south_africa,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
4,SRR14434437,south_africa,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
5,SRR14434438,south_africa,G,C,T,T,C,T,T,T,...,N,N,N,N,N,N,N,N,N,N
6,SRR14434439,south_africa,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
7,SRR14434441,south_africa,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
8,SRR14434442,south_africa,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
9,SRR14434443,south_africa,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N


In [87]:
df_senegal = preprocessing(snp_list="../data/snp.lists",
                      df_bio="../accessions/senegal/senegal_data.tsv",
                      df_sra="../work/c9/27675caa515b0799917d6d9f12e348/sra_senegal_data_biosamples.txt",
                      country="senegal")
df_senegal

genomic_position,sra_accession,country,480,504,567,646,681,744,759,1143,...,2635732,2957108,3279114,3353396,3412297,3843648,3882853,3918673,4283341,4349502
0,SRR10018352,senegal,G,T,A,T,C,C,A,A,...,N,N,N,N,N,N,N,N,N,N
1,SRR10018355,senegal,G,T,A,T,C,C,A,A,...,N,N,N,N,N,N,N,N,N,N
2,SRR10018395,senegal,G,T,A,T,C,C,A,A,...,N,N,N,N,N,N,N,N,N,N
3,SRR10018436,senegal,G,T,N,T,C,N,N,A,...,N,N,N,N,N,N,N,N,N,N
4,SRR10018441,senegal,G,T,N,T,C,N,N,A,...,N,N,N,N,N,N,N,N,N,N
5,SRR10018448,senegal,G,T,N,T,C,N,N,N,...,N,N,N,N,N,N,N,N,N,N
6,SRR10018690,senegal,N,N,N,T,N,C,A,A,...,N,N,N,N,N,N,N,N,N,N
7,SRR10665486,senegal,G,T,A,T,C,C,A,A,...,N,N,N,N,N,N,N,N,N,N
8,SRR10665490,senegal,G,T,N,T,C,N,N,A,...,N,N,N,N,N,N,N,N,N,N
9,SRR10665492,senegal,G,T,A,T,C,C,A,A,...,N,N,N,N,N,N,N,N,N,N


In [ ]:
df_nigeria_pig = preprocessing(snp_list="../data/snp.lists",
                      df_bio="../accessions/senegal/senegal_data.tsv",
                      df_sra="../work/dc/94fd1d5d43b6edfb9708c0904b207f/sra_filtered_nigeria_nigeria_pig_data_biosamples.txt",
                      country="nigeria")
df_nigeria_pig

In [178]:
df_kenya=pd.read_csv("../data/kenya_processed_salmonell.csv")
df_kenya

,Unnamed: 0,sra_accession,15,27,78,480,504,534,646,681,...,4839562,4841514,4841563,4843995,4851006,4853017,4855381,20462,65978,2914287
0,0,SRR10502163,T,A,A,G,T,C,T,C,...,N,N,N,N,N,N,N,N,N,N
1,1,SRR10502184,T,A,A,G,T,C,T,C,...,N,N,N,N,N,N,N,N,N,N
2,2,SRR12539197,N,N,N,G,N,C,T,N,...,N,N,N,N,N,N,N,N,N,N
3,3,SRR12539198,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
4,4,SRR12539199,N,N,N,G,N,C,T,N,...,N,N,N,N,N,N,N,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,127,SRR4243040,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
128,128,SRR4733853,N,N,N,G,N,C,T,N,...,N,N,N,N,N,N,N,N,N,N
129,129,SRR4733857,N,N,N,N,T,C,N,N,...,N,N,N,N,N,N,N,N,N,N
130,130,SRR5972858,N,N,N,G,N,C,T,N,...,T,C,C,T,T,T,T,N,N,N


In [206]:
df_kenya_snp_metadata= combine_metadata(df_snp=df_kenya,
                      df_bio="../accessions/kenya/kenya.tsv",
                                    
                      df_sra="../work/4c/b76ef2daa417d62c384b104f0f6016/sra_kenya_biosamples.txt",
                      country="kenya" 
                        )

,BioSample,Isolation source,sra_accession,Unnamed: 0,country,15,27,78,480,504,...,4839562,4841514,4841563,4843995,4851006,4853017,4855381,20462,65978,2914287
0,SAMN03576958,feces swine,SRR2163243,51,kenya,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
1,SAMN03576962,feces swine,SRR2163246,52,kenya,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
2,SAMN03576963,feces swine,SRR2163247,53,kenya,N,A,N,G,N,...,N,N,N,N,N,N,N,N,N,N
3,SAMN03576956,feces swine,SRR2163254,54,kenya,T,A,N,G,N,...,N,N,N,N,N,N,N,N,N,N
4,SAMN03576959,feces swine,SRR2163255,55,kenya,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,SAMN20456392,Kibera,SRR15369302,46,kenya,N,A,N,G,N,...,N,N,N,N,N,N,N,N,N,N
128,SAMN20456390,Kibera,SRR15369303,47,kenya,N,A,N,G,N,...,N,N,N,N,N,N,N,N,N,N
129,SAMN20456385,Kibera,SRR15369305,48,kenya,N,A,N,G,N,...,N,N,N,N,N,N,N,N,N,N
130,SAMN20456395,Kibera,SRR15369306,49,kenya,N,A,N,G,N,...,N,N,N,N,N,N,N,N,N,N


In [77]:
df_southafrica_snp_metadata= combine_metadata(df_snp=df_south_africa,
                      df_bio="../accessions/south_africa/south_africa.tsv",
                                
                        df_sra="../work/29/9d00016990fe8d0f623ba6840b6282/sra_south_africa_biosamples.txt",

                      country="south_africa" 
                        )

In [218]:
df_kenya_snp_metadata=df_kenya_snp_metadata.drop(columns=["Unnamed: 0"])
df_kenya_snp_metadata

,BioSample,Isolation source,sra_accession,country,15,27,78,480,504,534,...,4839562,4841514,4841563,4843995,4851006,4853017,4855381,20462,65978,2914287
0,SAMN03576958,feces swine,SRR2163243,kenya,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
1,SAMN03576962,feces swine,SRR2163246,kenya,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
2,SAMN03576963,feces swine,SRR2163247,kenya,N,A,N,G,N,C,...,N,N,N,N,N,N,N,N,N,N
3,SAMN03576956,feces swine,SRR2163254,kenya,T,A,N,G,N,C,...,N,N,N,N,N,N,N,N,N,N
4,SAMN03576959,feces swine,SRR2163255,kenya,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,SAMN20456392,Kibera,SRR15369302,kenya,N,A,N,G,N,C,...,N,N,N,N,N,N,N,N,N,N
128,SAMN20456390,Kibera,SRR15369303,kenya,N,A,N,G,N,C,...,N,N,N,N,N,N,N,N,N,N
129,SAMN20456385,Kibera,SRR15369305,kenya,N,A,N,G,N,C,...,N,N,N,N,N,N,N,N,N,N
130,SAMN20456395,Kibera,SRR15369306,kenya,N,A,N,G,N,C,...,N,N,N,N,N,N,N,N,N,N


In [63]:
df_nigeria_beef=pd.read_csv("../data/nigeria_processed(2).csv")
df_nigeria_beef

,BioSample,Isolation source,sra_accession,country,27,480,504,534,646,681,...,4564823,4564826,4564965,4564976,4564980,4564999,4565001,4565035,4565066,4661295
0,SAMN07163935,cattle stool (Bos taurus),SRR5627235,nigeria,A,G,N,C,T,C,...,N,N,N,N,N,N,N,N,N,N
1,SAMN07163938,cattle stool (Bos taurus),SRR5627238,nigeria,A,G,N,C,T,C,...,N,N,N,N,N,N,N,N,N,N
2,SAMN07163941,cattle stool (Bos taurus),SRR5627163,nigeria,A,G,T,C,T,C,...,N,N,N,N,N,N,N,N,N,N
3,SAMN07163937,cattle stool (Bos taurus),SRR5627236,nigeria,A,G,T,C,N,N,...,N,N,N,N,N,N,N,N,N,N
4,SAMN07163936,cattle stool (Bos taurus),SRR5627241,nigeria,N,G,T,C,T,C,...,N,N,N,N,N,N,N,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,SAMN31852854,beef,SRR22395760,nigeria,A,N,N,C,T,N,...,N,N,N,N,N,N,N,N,N,N
100,SAMN31852864,beef,SRR22395764,nigeria,N,G,N,C,T,N,...,N,N,N,N,N,N,N,N,N,N
101,SAMN31852862,beef,SRR22395766,nigeria,A,N,N,C,T,C,...,N,N,N,N,N,N,N,N,N,N
102,SAMN31852861,beef,SRR22395767,nigeria,A,G,N,C,N,N,...,N,N,N,N,N,N,N,N,N,N


In [67]:
df_nigeria_beef=df_nigeria_beef.drop(columns=["BioSample","Isolation source"])

In [68]:
df_nigeria_beef_metadata= combine_metadata(df_snp=df_nigeria_beef,
                      df_bio="../accessions/nigeria/bovine/filtered_nigeria_nigeria_beef_data.tsv",                                  
                      df_sra="../work/a2/6e9f4ac192144d46f029eb900dd19b/sra_filtered_nigeria_nigeria_beef_data_biosamples.txt",
                      country="nigeria" 
                        )
df_nigeria_beef_metadata

,BioSample,Isolation source,sra_accession,country,27,480,504,534,646,681,...,4564823,4564826,4564965,4564976,4564980,4564999,4565001,4565035,4565066,4661295
0,SAMN07163935,cattle stool (Bos taurus),SRR5627235,nigeria,A,G,N,C,T,C,...,N,N,N,N,N,N,N,N,N,N
1,SAMN07163938,cattle stool (Bos taurus),SRR5627238,nigeria,A,G,N,C,T,C,...,N,N,N,N,N,N,N,N,N,N
2,SAMN07163941,cattle stool (Bos taurus),SRR5627163,nigeria,A,G,T,C,T,C,...,N,N,N,N,N,N,N,N,N,N
3,SAMN07163937,cattle stool (Bos taurus),SRR5627236,nigeria,A,G,T,C,N,N,...,N,N,N,N,N,N,N,N,N,N
4,SAMN07163936,cattle stool (Bos taurus),SRR5627241,nigeria,N,G,T,C,T,C,...,N,N,N,N,N,N,N,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,SAMN31852854,beef,SRR22395760,nigeria,A,N,N,C,T,N,...,N,N,N,N,N,N,N,N,N,N
100,SAMN31852864,beef,SRR22395764,nigeria,N,G,N,C,T,N,...,N,N,N,N,N,N,N,N,N,N
101,SAMN31852862,beef,SRR22395766,nigeria,A,N,N,C,T,C,...,N,N,N,N,N,N,N,N,N,N
102,SAMN31852861,beef,SRR22395767,nigeria,A,G,N,C,N,N,...,N,N,N,N,N,N,N,N,N,N


In [88]:
df_senegal_metadata= combine_metadata(df_snp=df_senegal,
                    df_bio="../accessions/senegal/senegal_data.tsv",
                      df_sra="../work/c9/27675caa515b0799917d6d9f12e348/sra_senegal_data_biosamples.txt",
                      country="senegal")
df_senegal_metadata

,BioSample,Isolation source,sra_accession,country,480,504,567,646,681,744,...,2635732,2957108,3279114,3353396,3412297,3843648,3882853,3918673,4283341,4349502
0,SAMN12056294,broiler chicken,SRR9301024,senegal,G,T,A,T,C,C,...,N,N,N,N,N,N,N,N,N,N
1,SAMN12056293,broiler chicken,SRR9301027,senegal,N,N,N,T,N,C,...,N,N,N,N,N,N,N,N,N,N
2,SAMN12056318,broiler chicken,SRR9301039,senegal,G,T,A,T,C,C,...,N,N,N,N,N,N,N,N,N,N
3,SAMN12056331,broiler chicken,SRR9301043,senegal,G,T,N,T,C,N,...,N,N,N,N,N,N,N,N,N,N
4,SAMN12056335,broiler chicken,SRR9301054,senegal,N,T,N,N,N,C,...,N,N,N,N,N,N,N,N,N,N
5,SAMN12056343,broiler chicken,SRR9301060,senegal,G,T,N,T,C,N,...,N,N,N,N,N,N,N,N,N,N
6,SAMN12056361,broiler chicken,SRR9301062,senegal,G,T,N,T,C,N,...,N,N,N,N,N,N,N,N,N,N
7,SAMN12056362,broiler chicken,SRR9301069,senegal,G,N,N,N,N,C,...,N,N,N,N,N,N,N,N,N,N
8,SAMN12056355,broiler chicken,SRR9301077,senegal,G,T,N,T,C,N,...,N,N,N,N,N,N,N,N,N,N
9,SAMN12056359,broiler chicken,SRR9301079,senegal,N,N,N,T,N,C,...,N,N,N,N,N,N,N,N,N,N


In [55]:
df_nigeria_pig_metadata= combine_metadata(df_snp=df_nigeria_pig,
                      df_bio="../accessions/nigeria/pig/filtered_nigeria_nigeria_pig_data.tsv",
                      df_sra="../work/dc/94fd1d5d43b6edfb9708c0904b207f/sra_filtered_nigeria_nigeria_pig_data_biosamples.txt",
                      country="nigeria" 
                        )
df_nigeria_pig_metadata

,BioSample,Isolation source,sra_accession,country,534,646,681,765,793,796,...,4738657,4738666,4738669,4738672,4738678,4743304,4751369,4751390,4751399,4777378
0,SAMN07135307,pig stool,SRR6950097,nigeria,C,T,C,T,C,T,...,N,N,N,N,N,N,N,N,N,N
1,SAMN07135306,pig stool,SRR6950133,nigeria,C,T,C,T,C,T,...,N,N,N,N,N,N,N,N,N,N
2,SAMN07135305,pig stool,SRR6950194,nigeria,C,T,C,T,C,T,...,N,N,N,N,N,N,N,N,N,N
3,SAMN07135304,pig stool,SRR6950215,nigeria,C,T,C,N,N,N,...,N,N,N,N,N,N,N,N,N,N
4,SAMN07135293,pig stool,SRR6950217,nigeria,C,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,SAMN07135187,pig stool,SRR7763516,nigeria,C,T,C,N,C,N,...,N,N,N,N,N,N,N,N,N,N
87,SAMN07135188,pig stool,SRR7763517,nigeria,C,T,C,T,C,T,...,N,N,N,N,N,N,N,N,N,N
88,SAMN09929385,pig stool,SRR7763447,nigeria,C,T,N,T,C,T,...,N,N,N,N,N,N,N,N,N,N
89,SAMN09929405,pig stool,SRR7763451,nigeria,C,N,C,T,C,T,...,N,N,N,N,N,N,N,N,N,N


In [9]:
df_nigeria_poulty_metadata= combine_metadata(df_snp=df_nigeria_paultry,
                      df_bio="../accessions/nigeria/filtered_nigeria_nigeria_poultry_data.tsv", 
                      df_sra="../work/c9/b58aedeb74c3574e9abd924de894b5/sra_filtered_nigeria_nigeria_poultry_data_biosamples.txt",
                     country="nigeria" 
                        )
df_nigeria_poulty_metadata

,BioSample,Isolation source,sra_accession,country,27,40,576,646,681,765,...,4486464,4745998,4746010,4746025,4746046,4746058,4746064,4746067,4746068,4746070
0,SAMN02368689,Poultry small intenstine,SRR1106403,nigeria,N,N,N,T,C,N,...,N,N,N,N,N,N,N,N,N,N
1,SAMN02368684,chicken,SRR1840690,nigeria,N,N,N,T,C,N,...,N,N,N,N,N,N,N,N,N,N
2,SAMN02368666,chicken,SRR1106413,nigeria,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
3,SAMN02368627,chicken,SRR1106442,nigeria,N,N,N,T,N,N,...,N,N,N,N,N,N,N,N,N,N
4,SAMN02368606,chicken,SRR1106456,nigeria,N,N,N,T,C,N,...,N,N,N,N,N,N,N,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,SAMEA6869863,Poultry vendoors and farms,ERR4183583,nigeria,N,N,N,T,N,N,...,N,N,N,N,N,N,N,N,N,N
80,SAMEA6869864,Poultry vendoors and farms,ERR4183584,nigeria,N,N,N,T,C,N,...,N,N,N,N,N,N,N,N,N,N
81,SAMEA6869865,Poultry vendoors and farms,ERR4183585,nigeria,A,A,A,T,C,T,...,N,N,N,N,N,N,N,N,N,N
82,SAMEA6869866,Poultry vendoors and farms,ERR4183586,nigeria,N,N,N,T,N,N,...,N,N,N,N,N,N,N,N,N,N


In [82]:
df_nigeria_paultry.head(50).value_counts()

In [220]:
# Drop rows with 'kibers' in the isolation column
df_kenya_snp_metadata_swine = df_kenya_snp_metadata[df_kenya_snp_metadata['Isolation source'] == 'feces swine']
# df_kenya_snp_metadata_swine=df_kenya_snp_metadata_swine.drop(columns=["Unnamed: 0"])

# df_kenya_snp_metadata =df_kenya_snp_metadata[df_kenya_snp_metadata['Isolation source'] != 'blood']
df_kenya_snp_metadata_swine

,BioSample,Isolation source,sra_accession,country,15,27,78,480,504,534,...,4839562,4841514,4841563,4843995,4851006,4853017,4855381,20462,65978,2914287
0,SAMN03576958,feces swine,SRR2163243,kenya,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
1,SAMN03576962,feces swine,SRR2163246,kenya,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
2,SAMN03576963,feces swine,SRR2163247,kenya,N,A,N,G,N,C,...,N,N,N,N,N,N,N,N,N,N
3,SAMN03576956,feces swine,SRR2163254,kenya,T,A,N,G,N,C,...,N,N,N,N,N,N,N,N,N,N
4,SAMN03576959,feces swine,SRR2163255,kenya,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,SAMN03577020,feces swine,SRR2981160,kenya,N,A,N,G,N,C,...,N,N,N,N,N,N,N,N,N,N
69,SAMN03577015,feces swine,SRR2981161,kenya,N,N,N,G,N,C,...,N,N,N,N,N,N,N,N,N,N
70,SAMN03577005,feces swine,SRR2981162,kenya,N,A,N,G,N,C,...,N,N,N,N,N,N,N,N,N,N
71,SAMN03577026,feces swine,SRR3272062,kenya,N,N,N,G,N,C,...,N,N,N,N,N,N,N,N,N,N


In [221]:
df_kenya_snp_metadata_swine.to_csv("kenya_swine_processed.csv")

In [200]:
df_kenya_snp_metadata = df_kenya_snp_metadata[df_kenya_snp_metadata['Isolation source'] != 'feces swine']
df_kenya_snp_metadata

,BioSample,Isolation source,sra_accession,Unnamed: 0,country,15,27,78,480,504,...,4839562,4841514,4841563,4843995,4851006,4853017,4855381,20462,65978,2914287
73,SAMN02843543,nile perch fillet,SRR3747653,121,kenya,N,A,N,G,N,...,N,N,N,N,N,N,N,N,N,N
74,SAMN02844775,beef masala spice mixture,SRR3747672,122,kenya,N,A,N,G,N,...,N,N,N,N,N,N,N,N,N,N
75,SAMN05340950,pilau masala spice mixture,SRR3924533,123,kenya,N,A,N,N,T,...,N,N,N,N,N,N,N,N,N,N
76,SAMN05340951,pilau masala spice mixture,SRR3924536,124,kenya,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
77,SAMN05944940,swine feces,SRR4733853,128,kenya,N,N,N,G,N,...,N,N,N,N,N,N,N,N,N,N
78,SAMN05944941,swine feces,SRR4733857,129,kenya,N,N,N,N,T,...,N,N,N,N,N,N,N,N,N,N
79,SAMN07279146,macademia nut,SRR6958628,131,kenya,N,N,N,G,N,...,T,C,C,T,T,T,T,C,T,G
80,SAMN07279146,macademia nut,SRR5972858,130,kenya,N,N,N,G,N,...,T,C,C,T,T,T,T,N,N,N
81,SAMN07164055,blood,SRR12539199,4,kenya,N,N,N,G,N,...,N,N,N,N,N,N,N,N,N,N
82,SAMN07164056,blood,SRR12539197,2,kenya,N,N,N,G,N,...,N,N,N,N,N,N,N,N,N,N


In [235]:
df_nigeria_paultry=pd.read_csv("../data/nigeria_processed.csvv")
df_nigeria_paultry

,BioSample,Isolation source,sra_accession,country,27,480,504,534,646,681,...,4564823,4564826,4564965,4564976,4564980,4564999,4565001,4565035,4565066,4661295
0,SAMN07163935,cattle stool (Bos taurus),SRR5627235,nigeria,A,G,N,C,T,C,...,N,N,N,N,N,N,N,N,N,N
1,SAMN07163938,cattle stool (Bos taurus),SRR5627238,nigeria,A,G,N,C,T,C,...,N,N,N,N,N,N,N,N,N,N
2,SAMN07163941,cattle stool (Bos taurus),SRR5627163,nigeria,A,G,T,C,T,C,...,N,N,N,N,N,N,N,N,N,N
3,SAMN07163937,cattle stool (Bos taurus),SRR5627236,nigeria,A,G,T,C,N,N,...,N,N,N,N,N,N,N,N,N,N
4,SAMN07163936,cattle stool (Bos taurus),SRR5627241,nigeria,N,G,T,C,T,C,...,N,N,N,N,N,N,N,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,SAMN31852854,beef,SRR22395760,nigeria,A,N,N,C,T,N,...,N,N,N,N,N,N,N,N,N,N
100,SAMN31852864,beef,SRR22395764,nigeria,N,G,N,C,T,N,...,N,N,N,N,N,N,N,N,N,N
101,SAMN31852862,beef,SRR22395766,nigeria,A,N,N,C,T,C,...,N,N,N,N,N,N,N,N,N,N
102,SAMN31852861,beef,SRR22395767,nigeria,A,G,N,C,N,N,...,N,N,N,N,N,N,N,N,N,N


In [237]:
df_nigeria_paultry

,BioSample,Isolation source,sra_accession,country,27,480,504,534,646,681,...,4564823,4564826,4564965,4564976,4564980,4564999,4565001,4565035,4565066,4661295
0,SAMN07163935,cattle stool (Bos taurus),SRR5627235,nigeria,A,G,N,C,T,C,...,N,N,N,N,N,N,N,N,N,N
1,SAMN07163938,cattle stool (Bos taurus),SRR5627238,nigeria,A,G,N,C,T,C,...,N,N,N,N,N,N,N,N,N,N
2,SAMN07163941,cattle stool (Bos taurus),SRR5627163,nigeria,A,G,T,C,T,C,...,N,N,N,N,N,N,N,N,N,N
3,SAMN07163937,cattle stool (Bos taurus),SRR5627236,nigeria,A,G,T,C,N,N,...,N,N,N,N,N,N,N,N,N,N
4,SAMN07163936,cattle stool (Bos taurus),SRR5627241,nigeria,N,G,T,C,T,C,...,N,N,N,N,N,N,N,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,SAMN31852854,beef,SRR22395760,nigeria,A,N,N,C,T,N,...,N,N,N,N,N,N,N,N,N,N
100,SAMN31852864,beef,SRR22395764,nigeria,N,G,N,C,T,N,...,N,N,N,N,N,N,N,N,N,N
101,SAMN31852862,beef,SRR22395766,nigeria,A,N,N,C,T,C,...,N,N,N,N,N,N,N,N,N,N
102,SAMN31852861,beef,SRR22395767,nigeria,A,G,N,C,N,N,...,N,N,N,N,N,N,N,N,N,N


In [242]:
# Merge all dataframes
df_all=pd.concat([df_nigeria_snp_metadata,df_nigeria_paultry,df_kenya_snp_metadata_swine])
df_all

,BioSample,Isolation source,sra_accession,country,27,480,504,534,646,681,...,4833823,4835094,4835514,4841514,4841563,4843995,4851006,4853017,4855381,20462
0,SAMN07163935,cattle stool (Bos taurus),SRR5627235,nigeria,A,G,N,C,T,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SAMN07163938,cattle stool (Bos taurus),SRR5627238,nigeria,A,G,N,C,T,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SAMN07163941,cattle stool (Bos taurus),SRR5627163,nigeria,A,G,T,C,T,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SAMN07163937,cattle stool (Bos taurus),SRR5627236,nigeria,A,G,T,C,N,N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SAMN07163936,cattle stool (Bos taurus),SRR5627241,nigeria,N,G,T,C,T,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,SAMN03577020,feces swine,SRR2981160,kenya,NaN,NaN,NaN,NaN,NaN,NaN,...,N,N,N,N,N,N,N,N,N,N
69,SAMN03577015,feces swine,SRR2981161,kenya,NaN,NaN,NaN,NaN,NaN,NaN,...,N,N,N,N,N,N,N,N,N,N
70,SAMN03577005,feces swine,SRR2981162,kenya,NaN,NaN,NaN,NaN,NaN,NaN,...,N,N,N,N,N,N,N,N,N,N
71,SAMN03577026,feces swine,SRR3272062,kenya,NaN,NaN,NaN,NaN,NaN,NaN,...,N,N,N,N,N,N,N,N,N,N


In [257]:
df_all.to_csv("training_data.csv")

# Assignment of  Source Categories

In [71]:
def category_allocation(df_data):
    # List of terms related to various categories
    bovine_terms = ['cattle stool','stool (Bos taurus)','intestine cattle','GOAT','GOAT MEAT','cattle lung', 'cattle', 'beef', 'cattle stool (Bos taurus)']
    poultry_terms = ['poultry feed', 'Poultry vendoors and farms', 'chicken heart', 'Poultry small intenstine',
                     'Poultry carcass', 'chicken liver', 'Chicken Spleen', 'chicken', 'poultry water']
    swine_terms=["feces swine","swine","swine stool","pig stool"]

    # Create the dictionary for replacement
    replacement_dict = {}
    for term in bovine_terms:
        replacement_dict[term] = 'bovine'
    for term in poultry_terms:
        replacement_dict[term] = 'poultry'
    for term in swine_terms:
        replacement_dict[term] = 'swine'

    # Function to replace terms based on the dictionary
    def replace_terms(description):
        for term, replacement in replacement_dict.items():
            if term in description:
                return replacement
        return description

    # Create the 'category' column using apply
    df_data['category'] = df_data['Isolation source'].apply(replace_terms)

    # Reorder the columns to insert 'category' as the second column
    column_order = ['Isolation source', 'category'] + [col for col in df_data.columns if col != 'Isolation source' and col != 'category']
    df_new = df_data[column_order]

    return df_new

In [10]:
df_ni_poultry=category_allocation(df_nigeria_poulty_metadata)
df_ni_poultry

,Isolation source,category,BioSample,sra_accession,country,27,40,576,646,681,...,4486464,4745998,4746010,4746025,4746046,4746058,4746064,4746067,4746068,4746070
0,Poultry small intenstine,poultry,SAMN02368689,SRR1106403,nigeria,N,N,N,T,C,...,N,N,N,N,N,N,N,N,N,N
1,chicken,poultry,SAMN02368684,SRR1840690,nigeria,N,N,N,T,C,...,N,N,N,N,N,N,N,N,N,N
2,chicken,poultry,SAMN02368666,SRR1106413,nigeria,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
3,chicken,poultry,SAMN02368627,SRR1106442,nigeria,N,N,N,T,N,...,N,N,N,N,N,N,N,N,N,N
4,chicken,poultry,SAMN02368606,SRR1106456,nigeria,N,N,N,T,C,...,N,N,N,N,N,N,N,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,Poultry vendoors and farms,poultry,SAMEA6869863,ERR4183583,nigeria,N,N,N,T,N,...,N,N,N,N,N,N,N,N,N,N
80,Poultry vendoors and farms,poultry,SAMEA6869864,ERR4183584,nigeria,N,N,N,T,C,...,N,N,N,N,N,N,N,N,N,N
81,Poultry vendoors and farms,poultry,SAMEA6869865,ERR4183585,nigeria,A,A,A,T,C,...,N,N,N,N,N,N,N,N,N,N
82,Poultry vendoors and farms,poultry,SAMEA6869866,ERR4183586,nigeria,N,N,N,T,N,...,N,N,N,N,N,N,N,N,N,N


In [59]:
df_ni_pig=category_allocation(df_nigeria_pig_metadata)
df_ni_pig

,Isolation source,category,BioSample,sra_accession,country,534,646,681,765,793,...,4738657,4738666,4738669,4738672,4738678,4743304,4751369,4751390,4751399,4777378
0,pig stool,swine,SAMN07135307,SRR6950097,nigeria,C,T,C,T,C,...,N,N,N,N,N,N,N,N,N,N
1,pig stool,swine,SAMN07135306,SRR6950133,nigeria,C,T,C,T,C,...,N,N,N,N,N,N,N,N,N,N
2,pig stool,swine,SAMN07135305,SRR6950194,nigeria,C,T,C,T,C,...,N,N,N,N,N,N,N,N,N,N
3,pig stool,swine,SAMN07135304,SRR6950215,nigeria,C,T,C,N,N,...,N,N,N,N,N,N,N,N,N,N
4,pig stool,swine,SAMN07135293,SRR6950217,nigeria,C,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,pig stool,swine,SAMN07135187,SRR7763516,nigeria,C,T,C,N,C,...,N,N,N,N,N,N,N,N,N,N
87,pig stool,swine,SAMN07135188,SRR7763517,nigeria,C,T,C,T,C,...,N,N,N,N,N,N,N,N,N,N
88,pig stool,swine,SAMN09929385,SRR7763447,nigeria,C,T,N,T,C,...,N,N,N,N,N,N,N,N,N,N
89,pig stool,swine,SAMN09929405,SRR7763451,nigeria,C,N,C,T,C,...,N,N,N,N,N,N,N,N,N,N


In [72]:
df_ni_beef=category_allocation(df_nigeria_beef_metadata)
df_ni_beef

,Isolation source,category,BioSample,sra_accession,country,27,480,504,534,646,...,4564823,4564826,4564965,4564976,4564980,4564999,4565001,4565035,4565066,4661295
0,cattle stool (Bos taurus),bovine,SAMN07163935,SRR5627235,nigeria,A,G,N,C,T,...,N,N,N,N,N,N,N,N,N,N
1,cattle stool (Bos taurus),bovine,SAMN07163938,SRR5627238,nigeria,A,G,N,C,T,...,N,N,N,N,N,N,N,N,N,N
2,cattle stool (Bos taurus),bovine,SAMN07163941,SRR5627163,nigeria,A,G,T,C,T,...,N,N,N,N,N,N,N,N,N,N
3,cattle stool (Bos taurus),bovine,SAMN07163937,SRR5627236,nigeria,A,G,T,C,N,...,N,N,N,N,N,N,N,N,N,N
4,cattle stool (Bos taurus),bovine,SAMN07163936,SRR5627241,nigeria,N,G,T,C,T,...,N,N,N,N,N,N,N,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,beef,bovine,SAMN31852854,SRR22395760,nigeria,A,N,N,C,T,...,N,N,N,N,N,N,N,N,N,N
100,beef,bovine,SAMN31852864,SRR22395764,nigeria,N,G,N,C,T,...,N,N,N,N,N,N,N,N,N,N
101,beef,bovine,SAMN31852862,SRR22395766,nigeria,A,N,N,C,T,...,N,N,N,N,N,N,N,N,N,N
102,beef,bovine,SAMN31852861,SRR22395767,nigeria,A,G,N,C,N,...,N,N,N,N,N,N,N,N,N,N


In [73]:
df_ni_beef.to_csv("../data/nigeria_bovine_metadata.csv",index=False)

In [74]:
df_ni_beef["category"].value_counts()

category
bovine    104
Name: count, dtype: int64

# Use allelle frequency conservation  for Dimentional Reduction

In [89]:
non_constant(df_kenya)

genomic_position,country
0,kenya
1,kenya
2,kenya
3,kenya
4,kenya
...,...
71,kenya
72,kenya
73,kenya
74,kenya


# Nigeria Dataset Processing


In [35]:
# Nigeria
# load data
df_nigeria=pd.read_table("../data/nigeria.tsv")
df_nigeria.head()

,#Organism group,Strain,Isolate identifiers,Serovar,Isolate,Create date,Location,Isolation source,Isolation type,SNP cluster,Min-same,Min-diff,BioSample,Assembly,AMR genotypes,Computed types
0,Salmonella enterica,RKS304_601-84,"BCW_2606,""CFSAN005617"",""RKS304_601-84"",""SRS536...","1,40:z35:-",PDT000002955.5,2020-04-27T12:13:59Z,Nigeria,food,environmental/other,NaN,NaN,NaN,SAMN02368165,NaN,mdsB=COMPLETE,"antigen_formula=40:z35:-,serotype=[1],40:z35:-"
1,Salmonella enterica,33(3),"33(3),""BCW_2809"",""SRS538850""",Uganda,PDT000003053.4,2015-12-12T15:21:45Z,Nigeria,"Human, faeces",clinical,PDS000001850.5,NaN,0.0,SAMN02368677,GCA_010639825.1,"mdsA=COMPLETE,mdsB=COMPLETE","antigen_formula=3,10:l,z13:1,5,serotype=Uganda"
2,Salmonella enterica,FMA0095,"FMA0095,""SRS565883""",Ealing,PDT000003898.3,2015-02-11T02:07:39Z,Nigeria,melon seed,environmental/other,NaN,NaN,NaN,SAMN02345045,GCA_009997545.1,"mdsA=COMPLETE,mdsB=COMPLETE","antigen_formula=35:g,m,s:-,serotype=Ealing"
3,Salmonella enterica,BCW_2640,"9(3),""BCW_2640"",""SRS533427""",Ank,PDT000047946.4,2015-07-23T05:10:39Z,Nigeria,Cattle liver,environmental/other,NaN,NaN,NaN,SAMN02368530,GCA_010369405.1,"mdsA=COMPLETE,mdsB=COMPLETE","antigen_formula=28:k:e,n,z15,serotype=Ank"
4,Salmonella enterica,FDA904406 1-1,"CFSAN032276,""FDA00008919"",""FDA904406 1-1"",""SRS...",Pomona,PDT000060513.2,2015-04-18T14:10:49Z,Nigeria,ground chili pepper,environmental/other,PDS000031411.1,1.0,NaN,SAMN03483118,GCA_011409935.1,"mdsA=COMPLETE,mdsB=COMPLETE","antigen_formula=28:y:1,7,serotype=Pomona"


In [60]:
# Extract Paultry from nigeria dataset
# Load the data
# Filter the DataFrame
filtered_df = df_nigeria[(df_nigeria['Isolation source'].notnull()) & (~df_nigeria['Isolation source'].isin(['feces', 'stool']))]
 # Assuming df_nigeria is your original DataFrame
filtered_poultry = df_nigeria[df_nigeria['Isolation source'].isin(['poultry feed','Poultry vendoors and farms','chicken heart','Poultry small intenstine','Poultry carcass','chicken liver','Chicken Spleen','chicken','poultry water'])]                             
# Display the filtered DataFrame
filtered_poultry.to_csv("../accessions/nigeria/filtered_nigeria_nigeria_poultry_data.tsv", sep='\t', index=False)
filtered_poultry

,#Organism group,Strain,Isolate identifiers,Serovar,Isolate,Create date,Location,Isolation source,Isolation type,SNP cluster,Min-same,Min-diff,BioSample,Assembly,AMR genotypes,Computed types
88,Salmonella enterica,BCW_2822,"66(3),""BCW_2822"",""SRS528782""",Wilhelmsburg,PDT000197317.1,2017-03-22T14:56:21Z,Nigeria,Poultry small intenstine,environmental/other,NaN,NaN,NaN,SAMN02368689,GCA_002062955.1,"mdsA=COMPLETE,mdsB=COMPLETE","antigen_formula=4:z38:-,serotype=Wilhelmsburg"
89,Salmonella enterica,BCW_2817,"BCW_2817,""KR 16"",""SRS865334""",Virchow,PDT000197326.1,2017-03-22T15:02:25Z,Nigeria,chicken,environmental/other,PDS000031736.79,22.0,16.0,SAMN02368684,GCA_002063155.1,"dfrA5=COMPLETE,gyrA_S83Y=POINT,mdsA=COMPLETE,m...","antigen_formula=7:r:1,2,serotype=Virchow"
92,Salmonella enterica,BCW_2794,"BCW_2794,""KR 225"",""SRS528772""",Bredeney,PDT000197354.1,2017-03-22T15:10:09Z,Nigeria,chicken,environmental/other,NaN,NaN,NaN,SAMN02368666,GCA_002063755.1,"blaTEM-1=COMPLETE,gyrA_D87N=POINT,mdsA=COMPLET...","antigen_formula=4:l,v:1,7,serotype=Bredeney"
94,Salmonella enterica,BCW_2750,"BCW_2750,""KR167"",""SRS528743""",Onireke,PDT000197377.1,2017-03-22T15:18:10Z,Nigeria,chicken,environmental/other,PDS000001391.75,8.0,14.0,SAMN02368627,GCA_002064225.1,"aadA1=COMPLETE,dfrA1=COMPLETE,gyrA_D87G=POINT,...","antigen_formula=4:b:1,2,serotype=Paratyphi B v..."
95,Salmonella enterica,BCW_2729,"BCW_2729,""KR 84"",""SRS528729""",Mbandaka,PDT000197390.1,2017-03-22T15:21:50Z,Nigeria,chicken,environmental/other,NaN,NaN,NaN,SAMN02368606,GCA_002064495.1,"aadA1=COMPLETE,dfrA15=COMPLETE,gyrA_D87Y=POINT...","antigen_formula=7:z10:e,n,z15,serotype=Mbandaka"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
682,Salmonella enterica,NaN,"D38,""ERS4597431"",""Salmonella""",NaN,PDT001291149.1,2022-04-20T13:38:33Z,Nigeria,Poultry vendoors and farms,NaN,PDS000019581.6,NaN,NaN,SAMEA6869863,GCA_023070735.1,"mdsA=COMPLETE,mdsB=COMPLETE,qnrB19=COMPLETE","antigen_formula=4:z10:1,5,serotype=Ituri"
683,Salmonella enterica,NaN,"D39,""ERS4597432"",""Salmonella""",NaN,PDT001291150.1,2022-04-20T13:38:33Z,Nigeria,Poultry vendoors and farms,NaN,PDS000108531.2,NaN,NaN,SAMEA6869864,GCA_023070715.1,"mdsA=COMPLETE,mdsB=COMPLETE","antigen_formula=7:r:1,2,serotype=Virchow"
684,Salmonella enterica,NaN,"D40,""ERS4597433"",""Salmonella""",NaN,PDT001291151.1,2022-04-20T13:38:33Z,Nigeria,Poultry vendoors and farms,NaN,PDS000108534.1,NaN,NaN,SAMEA6869865,GCA_023070695.1,"fosA7=COMPLETE,fosB=PARTIAL_END_OF_CONTIG,mdsA...","antigen_formula=4:z4,z23:1,2,serotype=Stanleyv..."
685,Salmonella enterica,NaN,"D41,""ERS4597434"",""Salmonella""",NaN,PDT001291152.1,2022-04-20T13:38:34Z,Nigeria,Poultry vendoors and farms,NaN,NaN,NaN,NaN,SAMEA6869866,GCA_023070635.1,"mdsA=COMPLETE,mdsB=COMPLETE","antigen_formula=7:g,s,t:-,serotype=Menston"


In [96]:
# extract cattle from nigeria dataset
filtered_beef = df_nigeria[df_nigeria['Isolation source'].isin(['cattle stool','intestine cattle','cattle lung','cattle','beef','cattle stool (Bos taurus)','GOAT MEAT','stool (Bos taurus)'])]                             

filtered_beef.to_csv("../accessions/nigeria/bovine/filtered_nigeria_nigeria_beef_data.tsv", sep='\t', index=False)
filtered_beef

,#Organism group,Strain,Isolate identifiers,Serovar,Isolate,Create date,Location,Isolation source,Isolation type,SNP cluster,Min-same,Min-diff,BioSample,Assembly,AMR genotypes,Computed types
117,Salmonella enterica,CFSAN059903,"CFSAN059903,""ID 85"",""SRS2238144""",Carno,PDT000236024.2,2021-06-07T16:09:27Z,Nigeria,cattle stool (Bos taurus),environmental/other,PDS000019592.8,5.0,7.0,SAMN07163935,GCA_002265865.2,"mdsA=COMPLETE,mdsB=COMPLETE","antigen_formula=8:z4,z23:-,serotype=Corvallis"
118,Salmonella enterica,CFSAN059900,"AK 587,""CFSAN059900"",""SRS2238147""",Ealing,PDT000236025.1,2017-08-23T00:01:32Z,Nigeria,cattle stool (Bos taurus),environmental/other,NaN,NaN,NaN,SAMN07163938,GCA_002265925.1,"mdsA=COMPLETE,mdsB=COMPLETE","antigen_formula=35:g,m,s:-,serotype=Ealing"
119,Salmonella enterica,CFSAN059897,"CFSAN059897,""DE 126"",""SRS2238072""",Altendorf,PDT000236026.1,2017-08-23T00:01:39Z,Nigeria,cattle stool (Bos taurus),environmental/other,NaN,NaN,NaN,SAMN07163941,GCA_002265935.1,"fosA7=MISTRANSLATION,mdsA=COMPLETE,mdsB=COMPLETE","antigen_formula=4:c:1,7,serotype=Altendorf"
120,Salmonella enterica,CFSAN059901,"CFSAN059901,""JM 580"",""SRS2238145""",Glostrup,PDT000236027.1,2017-08-23T00:02:08Z,Nigeria,cattle stool (Bos taurus),environmental/other,NaN,NaN,NaN,SAMN07163937,GCA_002265965.1,"mdsA=COMPLETE,mdsB=COMPLETE","antigen_formula=8:z10:e,n,z15,serotype=Glostru..."
121,Salmonella enterica,CFSAN059902,"1D 80,""CFSAN059902"",""SRS2238150""",Ekotedo,PDT000236028.1,2017-08-23T00:02:09Z,Nigeria,cattle stool (Bos taurus),environmental/other,NaN,NaN,NaN,SAMN07163936,GCA_002265975.1,"mdsA=COMPLETE,mdsB=COMPLETE","antigen_formula=9,46:z4,z23:-,serotype=Ekotedo"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
783,Salmonella enterica,NMSU-KFB83,"KFB 83,""NMSU-KFB83"",""SRS15852141""",NaN,PDT001496393.1,2022-11-23T19:45:08Z,Nigeria,beef,environmental/other,NaN,NaN,NaN,SAMN31852854,GCA_026363035.1,"mdsA=COMPLETE,mdsB=COMPLETE","antigen_formula=16:l,v:-,serotype=I 16:l,v:-"
786,Salmonella enterica,NMSU-KFB131,"KFB 131,""KFB114.131_002"",""NMSU-KFB131"",""SRS158...",NaN,PDT001496397.1,2022-11-23T19:45:09Z,Nigeria,beef,environmental/other,PDS000019571.5,17.0,42.0,SAMN31852864,GCA_026364535.1,"mdsA=COMPLETE,mdsB=COMPLETE","antigen_formula=9:g,p:-,serotype=Dublin"
788,Salmonella enterica,NMSU-KFB99,"KFB 99,""NMSU-KFB99"",""SRS15852136""",NaN,PDT001496399.1,2022-11-23T19:45:09Z,Nigeria,beef,environmental/other,NaN,NaN,NaN,SAMN31852862,GCA_026362995.1,"mdsA=COMPLETE,mdsB=COMPLETE","antigen_formula=28:y:1,5,serotype=Nima"
789,Salmonella enterica,NMSU-KFB98,"KFB 98,""NMSU-KFB98"",""SRS15852135""",NaN,PDT001496400.1,2022-11-23T19:45:10Z,Nigeria,beef,environmental/other,NaN,NaN,NaN,SAMN31852861,GCA_026362135.1,"mdsA=COMPLETE,mdsB=COMPLETE","antigen_formula=30:b:e,n,x,serotype=Urbana"


In [230]:
df_nig=pd.read_table("../accessions/nigeria/nigeria.tsv")
# extract pig from nigeria dataset
filtered_pig = df_nig[df_nig['Isolation source'].isin(['pig stool','pig'])]                             

filtered_pig.to_csv("../accessions/nigeria/pig/filtered_nigeria_nigeria_pig_data.tsv", sep='\t', index=False)
filtered_pig

,#Organism group,Strain,Isolate identifiers,Serovar,Isolate,Create date,Location,Isolation source,Isolation type,SNP cluster,Min-same,Min-diff,BioSample,Assembly,AMR genotypes core,Computed types
140,Salmonella enterica,CFSAN064163,"CFSAN064163,""KP-101"",""SRS3126453""",Elisabethville,PDT000303437.2,2018-04-05T14:43:40Z,Nigeria,pig stool,environmental/other,PDS000093403.5,3.0,38.0,SAMN07135279,GCA_010573885.1,NaN,"antigen_formula=3,10:r:1,7,serotype=Elisabethv..."
141,Salmonella enterica,CFSAN064135,"CFSAN064135,""KP-288"",""SRS3126457""",Amina,PDT000303438.2,2018-04-05T14:43:40Z,Nigeria,pig stool,environmental/other,NaN,NaN,NaN,SAMN07135236,GCA_010787395.1,NaN,"antigen_formula=16:i:1,5,serotype=Amina"
144,Salmonella enterica,CFSAN064160,"CFSAN064160,""KP-41"",""SRS3126514""",Elisabethville,PDT000303442.2,2018-04-05T14:43:41Z,Nigeria,pig stool,environmental/other,PDS000029271.2,2.0,NaN,SAMN07135282,GCA_010787375.1,NaN,"antigen_formula=3,10:r:1,7,serotype=Elisabethv..."
145,Salmonella enterica,CFSAN064136,"CFSAN064136,""KP-46"",""SRS3126547""",Badagry,PDT000303443.2,2018-04-05T14:43:41Z,Nigeria,pig stool,environmental/other,NaN,NaN,NaN,SAMN07135235,GCA_011153475.1,NaN,"antigen_formula=16:z10:1,5,serotype=Badagry"
148,Salmonella enterica,CFSAN064249,"CFSAN064249,""KP-301"",""SRS3126552""",Zongo,PDT000303447.2,2018-04-05T14:43:42Z,Nigeria,pig stool,environmental/other,NaN,NaN,NaN,SAMN07135180,GCA_010471865.1,NaN,"antigen_formula=3,10:z35:1,7,serotype=Zongo"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,Salmonella enterica,CFSAN064242,"CFSAN064242,""KP-125"",""SRS3721916""",Tennessee,PDT000367992.2,2018-08-29T00:08:59Z,Nigeria,pig stool,environmental/other,NaN,NaN,NaN,SAMN07135187,GCA_006790355.1,NaN,"antigen_formula=-:z29:-,serotype=I -:z29:-"
319,Salmonella enterica,CFSAN064241,"CFSAN064241,""KP-14"",""SRS3721917""",Tamberma,PDT000367993.1,2018-08-29T00:09:00Z,Nigeria,pig stool,environmental/other,NaN,NaN,NaN,SAMN07135188,GCA_004224545.1,NaN,"antigen_formula=47:z4,z24:-,serotype=Tamberma"
322,Salmonella enterica,CFSAN064218,"CFSAN064218,""KP-71"",""SRS3728700""",Larochelle,PDT000368614.1,2018-08-30T23:40:38Z,Nigeria,pig stool,environmental/other,PDS000029158.11,NaN,29.0,SAMN09929385,GCA_006521355.1,qnrB19=COMPLETE,"antigen_formula=7:e,h:1,2,serotype=Larochelle"
323,Salmonella enterica,CFSAN064217,"CFSAN064217,""KP-76"",""SRS3728704""",Lagos,PDT000368615.1,2018-08-30T23:40:39Z,Nigeria,pig stool,environmental/other,PDS000037348.2,NaN,26.0,SAMN09929405,GCA_006521395.1,NaN,"antigen_formula=4:i:1,5,serotype=Lagos"
